In [2]:
# === ERA5 -> Cold hazards (2010–2024) for Tajikistan: ONE-CELL NOTEBOOK ===
# Produit:
# 1) TNn_mean_2010_2024.tif  (moyenne multi-annuelle du minimum annuel de Tmin, en °C)
# 2) FD_mean_2010_2024.tif   (moyenne multi-annuelle de jours de gel/an, Tmin < 0°C)
import os, gc
from pathlib import Path
import numpy as np
import xarray as xr
import rioxarray  # pour .rio.to_raster & CRS
import cdsapi
import rasterio

# --- Config RAM/threads (plus stable) ---
os.environ.setdefault("OMP_NUM_THREADS", "1")
os.environ.setdefault("OPENBLAS_NUM_THREADS", "1")
os.environ.setdefault("MKL_NUM_THREADS", "1")
os.environ.setdefault("NUMEXPR_NUM_THREADS", "1")

# --- PARAMÈTRES ---
YEARS  = list(range(2010, 2025))       # 2010..2024
MONTHS = list(range(1, 13))            # Jan..Dec
AREA_TJK = [41.1, 67.3, 36.5, 75.2]    # [North, West, South, East] (ordre CDS)
ROOT = Path(".")
OUT_DIR = ROOT / "../data/hazards/cold"
TMP_DIR = ROOT / "./_tmp_era5_tjk_cold_allmonths"
OUT_DIR.mkdir(parents=True, exist_ok=True)
TMP_DIR.mkdir(parents=True, exist_ok=True)
OUT_TNN = OUT_DIR / "TNn_mean_2010_2024.tif"
OUT_FD  = OUT_DIR / "FD_mean_2010_2024.tif"

print("Output dir:", OUT_DIR.resolve())
print("Temp dir  :", TMP_DIR.resolve())

# --- Helpers dans la même cellule ---
def cds_client():
    # nécessite ~/.cdsapirc
    return cdsapi.Client()

def download_hourly_month(c, year: int, month: int, out_nc: Path, area_bbox):
    """Télécharge ERA5 horaire t2m (si non présent)."""
    if out_nc.exists():
        print(f"[skip] {out_nc.name}")
        return
    print(f"[download] ERA5 hourly t2m {year}-{month:02d}")
    c.retrieve(
        "reanalysis-era5-single-levels",
        {
            "product_type": "reanalysis",
            "variable": "2m_temperature",
            "year": str(year),
            "month": f"{month:02d}",
            "day": [f"{d:02d}" for d in range(1, 32)],
            "time": [f"{h:02d}:00" for h in range(24)],
            "area": area_bbox,  # [N, W, S, E]
            "format": "netcdf",
        },
        str(out_nc),
    )

def detect_time_dim(da: xr.DataArray) -> str:
    """Retourne 'time' ou 'valid_time' selon le NetCDF."""
    if "time" in da.dims: return "time"
    if "valid_time" in da.dims: return "valid_time"
    raise ValueError(f"Aucune dimension temporelle trouvée. Dims: {list(da.dims)}")

def ensure_spatial(da: xr.DataArray) -> xr.DataArray:
    """Fixe dims spatiales & CRS (WGS84) pour rioxarray."""
    if "latitude" not in da.dims or "longitude" not in da.dims:
        raise ValueError(f"Dims inattendues ERA5: {da.dims}")
    da = da.rio.set_spatial_dims(x_dim="longitude", y_dim="latitude", inplace=False)
    da = da.rio.write_crs("EPSG:4326", inplace=False)
    return da

# --- Traitement principal (mois par mois -> agrégation annuelle -> moyenne multi-annuelle) ---
c = cds_client()
tnn_years = []  # TNn par année
fd_years  = []  # Frost Days par année

for y in YEARS:
    print(f"\n=== YEAR {y} ===")
    tnn_year = None  # min des Tmin journaliers sur l'année
    fd_year  = None  # somme des jours (Tmin<0°C) sur l'année

    for m in MONTHS:
        hourly_nc = TMP_DIR / f"era5_t2m_hourly_tjk_{y}_{m:02d}.nc"
        download_hourly_month(c, y, m, hourly_nc, AREA_TJK)

        print(f"[process] {hourly_nc.name} -> daily Tmin & FD part")
        ds = xr.open_dataset(hourly_nc, chunks={"time": 240})
        if "t2m" not in ds:
            raise ValueError(f"'t2m' manquant dans {hourly_nc}")
        da_hour = (ds["t2m"] - 273.15).rename("t2m")  # K → °C

        tdim = detect_time_dim(da_hour)
        # Tmin quotidien
        da_daily_tmin = da_hour.resample({tdim: "1D"}).min(skipna=True)
        # FD du mois (nb jours Tmin < 0°C)
        fd_month = (da_daily_tmin < 0).sum(dim=tdim).astype("float32")
        # TNn du mois (min des Tmin quotidiens)
        tnn_month = da_daily_tmin.min(dim=tdim)

        # Accumulation annuelle
        tnn_year = tnn_month if tnn_year is None else xr.ufuncs.minimum(tnn_year, tnn_month)
        fd_year  = fd_month if fd_year  is None else (fd_year + fd_month)

        ds.close()
        del ds, da_hour, da_daily_tmin, fd_month, tnn_month
        gc.collect()

    # Fin d'année: spatial + cast
    tnn_year = ensure_spatial(tnn_year).astype("float32")
    fd_year  = ensure_spatial(fd_year).astype("float32")
    tnn_years.append(tnn_year)
    fd_years.append(fd_year)

    print(f"[year {y}] TNn range: {float(tnn_year.min()):.1f}..{float(tnn_year.max()):.1f} °C | "
          f"FD range: {float(fd_year.min()):.0f}..{float(fd_year.max()):.0f} days")

print("\n[aggregate] Multi-annual means (2010–2024)")
tnn_stack = xr.concat(tnn_years, dim="year").assign_coords(year=("year", YEARS))
fd_stack  = xr.concat(fd_years,  dim="year").assign_coords(year=("year", YEARS))
tnn_mean = tnn_stack.mean(dim="year", skipna=True)
fd_mean  = fd_stack.mean(dim="year",  skipna=True)

# Écriture GeoTIFF
tnn_mean.rio.to_raster(OUT_TNN)
fd_mean.rio.to_raster(OUT_FD)

# Sanity checks (min/mean/max)
def quick_stats(path):
    with rasterio.open(path) as src:
        arr = src.read(1)
        nod = src.nodata
    arr = arr.astype("float32")
    if nod is not None:
        mask = arr != nod
        vals = arr[mask]
    else:
        vals = arr[np.isfinite(arr)]
    return float(np.nanmin(vals)), float(np.nanmean(vals)), float(np.nanmax(vals))

print("\n✅ WROTE:")
print(" -", OUT_TNN.resolve())
print(" -", OUT_FD.resolve())
print("TNn_mean  (°C)  min/mean/max:", quick_stats(OUT_TNN))
print("FD_mean   (days) min/mean/max:", quick_stats(OUT_FD))


Output dir: /Users/nassimdekkar/Documents/GitHub/RI-Infra-exposure/data/hazards/cold
Temp dir  : /Users/nassimdekkar/Documents/GitHub/RI-Infra-exposure/notebooks/_tmp_era5_tjk_cold_allmonths

=== YEAR 2010 ===
[skip] era5_t2m_hourly_tjk_2010_01.nc
[process] era5_t2m_hourly_tjk_2010_01.nc -> daily Tmin & FD part
[skip] era5_t2m_hourly_tjk_2010_02.nc
[process] era5_t2m_hourly_tjk_2010_02.nc -> daily Tmin & FD part
[skip] era5_t2m_hourly_tjk_2010_03.nc
[process] era5_t2m_hourly_tjk_2010_03.nc -> daily Tmin & FD part
[skip] era5_t2m_hourly_tjk_2010_04.nc
[process] era5_t2m_hourly_tjk_2010_04.nc -> daily Tmin & FD part
[skip] era5_t2m_hourly_tjk_2010_05.nc
[process] era5_t2m_hourly_tjk_2010_05.nc -> daily Tmin & FD part
[skip] era5_t2m_hourly_tjk_2010_06.nc
[process] era5_t2m_hourly_tjk_2010_06.nc -> daily Tmin & FD part
[skip] era5_t2m_hourly_tjk_2010_07.nc
[process] era5_t2m_hourly_tjk_2010_07.nc -> daily Tmin & FD part
[skip] era5_t2m_hourly_tjk_2010_08.nc
[process] era5_t2m_hourly_tjk_2

2025-10-20 19:10:18,232 INFO Request ID is c4a2673d-32b0-4625-95f7-b1ad608eba06
2025-10-20 19:10:18,331 INFO status has been updated to accepted
2025-10-20 19:10:26,951 INFO status has been updated to running
2025-10-20 19:11:34,268 INFO status has been updated to successful


ccff0f7ba0511faf3a37df7fad097602.nc:   0%|          | 0.00/925k [00:00<?, ?B/s]

[process] era5_t2m_hourly_tjk_2016_08.nc -> daily Tmin & FD part
[download] ERA5 hourly t2m 2016-09


2025-10-20 19:11:35,479 INFO Request ID is 3a22402e-cb49-457b-aba3-46891b6f6f46
2025-10-20 19:11:35,551 INFO status has been updated to accepted
2025-10-20 19:11:49,238 INFO status has been updated to running
2025-10-20 19:11:56,986 INFO status has been updated to accepted
2025-10-20 19:12:08,438 INFO status has been updated to running
2025-10-20 19:13:29,961 INFO status has been updated to successful


...

af71dd52439bc162ec8e72e03844306.nc:   0%|          | 0.00/972k [00:00<?, ?B/s]

[process] era5_t2m_hourly_tjk_2024_11.nc -> daily Tmin & FD part
[download] ERA5 hourly t2m 2024-12


2025-10-20 21:56:59,404 INFO Request ID is b9911fe8-e018-4634-9918-f524a9b459a3
2025-10-20 21:56:59,478 INFO status has been updated to accepted
2025-10-20 21:57:13,060 INFO status has been updated to running
2025-10-20 21:57:32,238 INFO status has been updated to accepted
2025-10-20 21:57:49,422 INFO status has been updated to running
2025-10-20 21:58:15,146 INFO status has been updated to successful


bfdcc6a281f4e2ac3bc1fd00183019b9.nc:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

[process] era5_t2m_hourly_tjk_2024_12.nc -> daily Tmin & FD part
[year 2024] TNn range: -41.1..-3.1 °C | FD range: 16..364 days

[aggregate] Multi-annual means (2010–2024)

✅ WROTE:
 - /Users/nassimdekkar/Documents/GitHub/RI-Infra-exposure/data/hazards/cold/TNn_mean_2010_2024.tif
 - /Users/nassimdekkar/Documents/GitHub/RI-Infra-exposure/data/hazards/cold/FD_mean_2010_2024.tif
TNn_mean  (°C)  min/mean/max: (-41.9543571472168, -23.977365493774414, -6.608868598937988)
FD_mean   (days) min/mean/max: (22.0, 195.68365478515625, 364.3999938964844)


In [1]:
# -------------------
#  In case of the kernel is full
# -------------------
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"